In [ ]:
# Local execution
import os
import sys

working_dir = os.getcwd()
sys.path.insert(1, working_dir.replace(working_dir.split('/')[-1], ''))
print(sys.path)

In [ ]:
# Remote
#!git clone https://github.com/ferugit/JSALT-LALMs-tutorial.git
#!cd JSALT-LALMs-tutorial

In [ ]:
!pip install -r ../requirements.txt

In [ ]:
# Download Qwen2.5-0.5B model
#!../download_hf_model.sh

In [ ]:
# Download AF2 model: CLAP encoder, Audio Transformer and XATTN
#!./download_af2.sh "YOUR_TOKEN_HERE"

# CLAP: Audio Encoder

![CLAP Architecture](../assets/audioclip-arch.png)

In [ ]:
import os
import yaml
import json